# LSTM을 이용한 텍스트 생성
- '경마장에 있는 말이 뛰고 있다'
- '그의 말이 법이다'
- '가는 말이 고와야 오는 말이 곱다'

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [4]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [5]:
# 단어 집합 크기 설정
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 12


In [6]:
sequences = []
for line in text.split('\n'):
  encoded = t.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

print('학습에 사용할 샘픔의 개수: %d' % len(sequences))

학습에 사용할 샘픔의 개수: 11


In [7]:
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
max_len = max(len(s)for s in sequences)
print('샘플의 최대 길이: ', max_len)

샘플의 최대 길이:  6


In [8]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [9]:
X = sequences[:, :-1]
y = sequences[:, -1]

Y = to_categorical(y, num_classes=vocab_size)

### 모델 정의
- Embedding
- LSTM


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [11]:
model = Sequential([
                    Embedding(vocab_size, 5, input_length=max_len - 1),
                    LSTM(32),
                    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 5)              60        
_________________________________________________________________
lstm (LSTM)                  (None, 32)                4864      
_________________________________________________________________
dense (Dense)                (None, 12)                396       
Total params: 5,320
Trainable params: 5,320
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
history = model.fit(X, Y, epochs=200, verbose=0)

In [14]:
history.history['accuracy'][-1]

0.7272727489471436

### 모델 검증

In [15]:
def sentence_generation(model, t, current_word, n):
  init_word = current_word
  sentence = ''
  for _ in range(n):
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=5, padding='pre')
    result = np.argmax(model.predict(encoded), axis=-1)
    for word, index in t.word_index.items():
      if index == result:
        break
    current_word = current_word + ' ' + word
    sentence = sentence + ' ' + word
    
  sentence = init_word + sentence
  return sentence

### 모델 변화
- Embedding Vector 개수: [2, 4, 6, 8]
- LSTM 노드 개수: [24, 32]

In [16]:
def change_model(n_embed, n_lstm):
  model = Sequential([
                      Embedding(vocab_size, n_embed, input_length=max_len - 1),
                      LSTM(n_lstm),
                      Dense(vocab_size, activation='softmax')
  ])
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  history = model.fit(X, Y, epochs=200, verbose=0)
  print(f"Accuracy: {history.history['accuracy'][-1]:.4f}")

  print(sentence_generation(model, t, '경마장에', 3))
  print(sentence_generation(model, t, '그의', 2))
  print(sentence_generation(model, t, '가는', 5))

In [17]:
for n_embed in [2, 4, 6, 8]:
  for n_lstm in [24, 32]:
    print('===========================')
    print(f'n_embed={n_embed}, n_rnn={n_lstm}')
    change_model(n_embed, n_lstm)
    print()

n_embed=2, n_rnn=24
Accuracy: 0.4545
경마장에 말이 말이 말이
그의 말이 말이
가는 말이 말이 말이 말이 있다

n_embed=2, n_rnn=32
Accuracy: 0.4545
경마장에 말이 말이 말이
그의 말이 말이
가는 말이 말이 말이 말이 곱다

n_embed=4, n_rnn=24
Accuracy: 0.5455
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 말이 말이 있다 있다

n_embed=4, n_rnn=32
Accuracy: 0.6364
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 말이 말이 말이 뛰고

n_embed=6, n_rnn=24
Accuracy: 0.5455
경마장에 말이 말이 법이다
그의 말이 말이
가는 말이 말이 말이 곱다 곱다

n_embed=6, n_rnn=32
Accuracy: 0.6364
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 말이 오는 뛰고 있다

n_embed=8, n_rnn=24
Accuracy: 0.6364
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 말이 말이 곱다 곱다

n_embed=8, n_rnn=32
Accuracy: 0.8182
경마장에 말이 고와야 뛰고
그의 말이 말이
가는 말이 고와야 오는 말이 곱다

